When we create a chatbot, we want it to take into account past conversations. In this way, he can give more natural and meaningful answers. In this notebook, we will create a chatbot considering past conversations.

For this, we will create a new chain structure. This chain structure will keep the last input and conversation history.

In [22]:
from dotenv import load_dotenv

load_dotenv()

True

In [23]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI()

In [24]:
from langchain_community.document_loaders import WebBaseLoader
# Our purpose in this notebook is to chat with a website.

loader = WebBaseLoader("https://blog.langchain.dev/langgraph/")
docs = loader.load() # Load the documents from the website

In [25]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
# We must chunk the documents to send them to the OpenAI API. Because the maximum token limit is 4096.

In [26]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

embedding = OpenAIEmbeddings() # Embedding model to convert the documents to vectors. Embedding means converting the text to a vector.
# We will use the FAISS vector store to store the embeddings of the documents.
vector_store = FAISS.from_documents(documents, embedding)

In [27]:
retriever = vector_store.as_retriever() # retriever is a model that retrieves the most similar documents to a given query. 

In [28]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import MessagesPlaceholder
# We use MessagesPlaceholder to set a place for the message history.

prompt = ChatPromptTemplate.from_messages(
    [
        MessagesPlaceholder(variable_name="chat_history"), # Here we actually keep the previous conversations.
        ("user", "{input}"),
        ("user", "Given the above conversation generate a search query to look up in order to get information relevant to the conversation."), # Here we wrote this to help the model create a search query based on the conversation above.
    ]
)
# The above prompt is a template for the chatbot. It will generate a search query based on the conversation history.

Now let's create a chain that retrieves the conversation history and returns the documents.

In [29]:
from langchain.chains import create_history_aware_retriever

retriever_chain = create_history_aware_retriever(
    model,
    retriever,
    prompt,
)
# The above chain is a chain that retrieves the conversation history and returns the documents.

Now let's test the chain. We will create HumanMessage and AIMessage objects to test the chain.

In [30]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = [
    HumanMessage(content="Can I use LangGraph for my project?"),
    AIMessage(content="Yes, you can use LangGraph for your project."),
]
# above chat history is a conversation history.

In [31]:
retriever_chain.invoke(
    {
        "chat_history": chat_history,
        "input": "Tell me how?",
    }
)
# The above code (retriever) will generate a search query based on the conversation history and return the documents.

[Document(page_content='TL;DR: LangGraph is module built on top of LangChain to better enable creation of cyclical graphs, often needed for agent runtimes.Python RepoPython YouTube PlaylistJS RepoIntroductionOne of the things we highlighted in our LangChain v0.1 announcement was the introduction of a new library: LangGraph. LangGraph is built on top of LangChain and completely interoperable with the LangChain ecosystem. It adds new value primarily through the introduction of an easy way to create cyclical graphs. This is often useful when creating agent runtimes.In this blog post, we will first walk through the motivations for LangGraph. We will then cover the basic functionality it provides. We will then spotlight two agent runtimes we\'ve implemented already. We will then highlight a few of the common modifications to these runtimes we\'ve heard requests for, and examples of implementing those. We will finish with a preview of what we will be releasing next.MotivationOne of the big v

We'll add a new chain to continue the conversation.

In [32]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Answer the user's questions based on the below context: \n\n{context}"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("user", "{input}"),
    ]
)
# The above prompt is a template for the chatbot. It will generate an answer based on the conversation history.

In [33]:
from langchain.chains.combine_documents import create_stuff_documents_chain
# Create a chain for passing a list of Documents to a model

documents_chain = create_stuff_documents_chain(
    model,
    prompt,
)

In [34]:
from langchain.chains import create_retrieval_chain
# This method will receive the documents and forward them to the retrieval chain so that the model will respond according to the relevant document

retriever_chain = create_retrieval_chain(
    retriever_chain, # this retriever looks at past conversations and retrieves relevant documents.
    documents_chain, # this chain passes the documents to the model.
)

In [35]:
chat_history = [
    HumanMessage(content="Can I use LangGraph for my project?"),
    AIMessage(content="Yes, you can use LangGraph for your project."),
]

response = retriever_chain.invoke(
    {
        "chat_history": chat_history,
        "input": "Tell me how?",
    }
)

In [36]:
response

{'chat_history': [HumanMessage(content='Can I use LangGraph for my project?'),
  AIMessage(content='Yes, you can use LangGraph for your project.')],
 'input': 'Tell me how?',
 'context': [Document(page_content='TL;DR: LangGraph is module built on top of LangChain to better enable creation of cyclical graphs, often needed for agent runtimes.Python RepoPython YouTube PlaylistJS RepoIntroductionOne of the things we highlighted in our LangChain v0.1 announcement was the introduction of a new library: LangGraph. LangGraph is built on top of LangChain and completely interoperable with the LangChain ecosystem. It adds new value primarily through the introduction of an easy way to create cyclical graphs. This is often useful when creating agent runtimes.In this blog post, we will first walk through the motivations for LangGraph. We will then cover the basic functionality it provides. We will then spotlight two agent runtimes we\'ve implemented already. We will then highlight a few of the commo

In [38]:
response["answer"]

'To use LangGraph for your project, you can follow these steps:\n\n1. Define a StateGraph: Initialize a StateGraph class by passing in a state definition that represents a central state object updated over time.\n\n2. Add Nodes: Use the graph.add_node(name, value) syntax to add nodes to the graph. Nodes should be functions or LCEL runnable that accept input and output dictionary with keys of the State object to update.\n\n3. Add Edges: Connect nodes by adding edges. There are three types of edges: Starting Edge (for the initial node), Normal Edges (to define the order of node execution), and Conditional Edges (for branching based on conditions).\n\n4. Compile: Once the graph is defined, you can compile it into a runnable using graph.compile(). This runnable can be called using standard LangChain runnable methods like .invoke, .stream, .astream_log, etc.\n\n5. Use Agent Executor: You can recreate the LangChain AgentExecutor using LangGraph to modify the internals of the AgentExecutor mo